In [19]:
import pandas as pd
import numpy as np
import mysql.connector

In [20]:
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

In [21]:
df = pd.read_csv('master_table/data_invoce_topan.csv')


In [22]:
#df['outlet_code'] = df['outlet_code'].factorize()[0]
df['ratio_total_per_balance'] = df['cash_memo_balance_amount']/df['cash_memo_total_amount']
df

,outlet_code,cash_memo_balance_amount,cash_memo_total_amount,ratio_total_per_balance,product_price,quantity,DPD
0,97437,1894157.0,2049284.0,0.924302,286909.000001,2.0,6
1,96858,10404232.0,10404232.0,1.000000,54000.000000,72.0,0
2,97492,4568250.0,4718250.0,0.968209,169090.999992,240.0,1
3,97492,4568250.0,4718250.0,0.968209,169090.999992,240.0,5
4,97492,4568250.0,4718250.0,0.968209,169090.999992,240.0,5
5,97492,4568250.0,4718250.0,0.968209,169090.999992,240.0,7
6,97492,5524853.0,5524853.0,1.000000,338181.999984,480.0,7
7,97492,5524853.0,5524853.0,1.000000,338181.999984,480.0,10
8,96787,2648295.0,2697324.0,0.981823,298908.999998,3.0,1
9,96696,2335810.0,2335810.0,1.000000,602181.999998,5.0,0


In [23]:
class DataFrameSelector(BaseEstimator, TransformerMixin): 
    def __init__(self, attibute_names):
        self.attibute_names = attibute_names
    def fit(self, X, y=None):
        return(self)
    def transform(self, X): 
        return(X[self.attibute_names].values)

In [24]:
scaler = StandardScaler()
# scaled_values = scaler.fit_transform(housing[["housing_median_age"]]).reshape(-1)

In [25]:
%run CategoricalEncoder.py

In [26]:
# Make the dataframe smaller and train test split
from sklearn.model_selection import train_test_split

df = df[:10000]
train_set, test_set = train_test_split(df, test_size=0.2, random_state=1)
# print(type(train_set))
# print(type(df_test))

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

cat_attributes = ['outlet_code']
num_attributes = ['DPD','product_price','ratio_total_per_balance']

num_pipeline = Pipeline([
    ('selector',DataFrameSelector(num_attributes)),
#     ('imputer',Imputer(strategy="median")),
    ('std_scaler', StandardScaler()),])

cat_pipeline = Pipeline([
    ('selector',DataFrameSelector(cat_attributes)),
    ('label_binarizer',CategoricalEncoder(encoding="onehot-dense")),])

full_pipeline = FeatureUnion(transformer_list=[
    ('cat_pipeline',cat_pipeline),
    ('num_pipeline',num_pipeline),])

df_prepared = full_pipeline.fit_transform(train_set.copy())
df_labels = train_set["quantity"].copy()
df_prepared.shape


(8000, 63)

In [29]:
#pd.DataFrame(df_prepared)
# df_test = pd.DataFrame(df_test)

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(df_prepared, df_labels)

df_test_labels = test_set["quantity"]

df_test = full_pipeline.transform(test_set)

quantity_predictions = lin_reg.predict(df_test)
lin_mse = mean_squared_error(df_test_labels, quantity_predictions)
lin_rmse = np.sqrt(lin_mse)/(df['quantity'].max() - df['quantity'].min()) * 100
str(lin_rmse) + ' normalized percent error'

'3.401112589013291 normalized percent error'

In [36]:
pd.DataFrame(df_test)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.134635,0.431851,0.455997
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.752227,0.288483,0.455997
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.752227,-0.592216,0.455997
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.752227,-0.660486,0.455997
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.688821,-0.209899,0.455997
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.340499,-0.127973,0.455997
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.894685,-0.701446,0.455997
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.546363,-0.719882,-0.826638
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.752227,0.431851,-1.039100
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.134635,-0.609282,0.455997


In [34]:
compare = {'predicted':list(quantity_predictions),'examples':list(df_test_labels)}
pd.DataFrame(compare)

,predicted,examples
0,262.718750,480.0
1,21.601562,3.0
2,188.140625,240.0
3,225.492188,480.0
4,251.164062,288.0
5,65.218750,24.0
6,185.585938,60.0
7,242.265625,216.0
8,14.664062,36.0
9,129.101562,3.0


In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import numpy as np

def evaluate_model(dataset, model, model_name, params):
    
    train_set, test_set = train_test_split(dataset)
    train_prepared = full_pipeline.fit_transform(train_set.copy())
    train_labels = train_set["quantity"]
    
    test_prepared = full_pipeline.transform(test_set)
    test_labels = test_set["quantity"]
    
    clf = GridSearchCV(model, params, cv=3)
    clf.fit(train_prepared, train_labels)
    
    #quantity_predictions = lin_reg.predict(df_test)
    #lin_mse = mean_squared_error(df_test_labels, quantity_predictions)
    #test_score = np.sqrt(lin_mse)/(df['quantity'].max() - df['quantity'].min()) * 100 )
    
    time = np.mean(clf.cv_results_['mean_fit_time'])
#     print(clf.best_estimator_)
    cv_score = clf.best_score_ #cv_score = cross_val_score(model, dataset_dict['X'], dataset_dict['y'], cv=3)
    eval_dict = {}
    eval_dict['Dataset'] = dataset_dict['name']
    eval_dict['Model'] = model_name
    eval_dict['Test Score'] = test_score
    eval_dict['CV Score'] = cv_score
    eval_dict['Time'] = time
    return eval_dict

###
### YOUR CODE HERE
###
